All imports

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
stemmer = LancasterStemmer()
import datetime
import time

loading data for training

In [3]:
Path_for_csv='C:/Users/trainees/Desktop/AllData/data/Classificationtable3.csv' # with classifcation
training_data = []
import pandas as pd
data=pd.read_csv(Path_for_csv)
data = data[pd.notnull(data['tokenized_source'])]
data=data[data.Category != 'None']
for index,row in data.iterrows():
    training_data.append({"class":row["Category"], "sentence":row["tokenized_source"]})

In [4]:
print ("%s sentences of training data" % len(training_data))

1594 sentences of training data


Format data and find number of Documents, words and classes

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    w = nltk.word_tokenize(pattern['sentence'])
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remove duplicates
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words),"Words")
number_of_documents=len(documents)
number_of_classes=len(classes)
number_of_words=len(words)

1594 documents
3 classes ['Class_3', 'Class_1', 'Class_2']
41468 Words


Format according to Network.
i.e mathematically making matrix of words

In [7]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

print ("# words", len(words))
print ("# classes", len(classes))

# words 41468
# classes 3


To load for train and test and convert them to array instead of numbers

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training, output, test_size=0.05, random_state=62)
import numpy as np
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)
X_test=np.asarray(X_test)
y_test=np.asarray(y_test)
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_train /= 255
y_train /= 255

One of the possible simple model with just 2 layers

In [10]:
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(number_of_words,)))
# model.add(Dense(3, activation='softmax'))

A different model with 5 layers

In [11]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(number_of_words,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(number_of_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               21232128  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
__________

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Start its training 

In [18]:
start_time = time.time()
history = model.fit(X_train, y_train,
                    batch_size=20,
                    epochs=10,
                    verbose=2,
                    validation_data=(X_test, y_test))
elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Train on 1514 samples, validate on 80 samples
Epoch 1/10
 - 21s - loss: 0.0041 - acc: 0.4392 - val_loss: 3.4481 - val_acc: 0.7000
Epoch 2/10
 - 22s - loss: 0.0037 - acc: 0.5746 - val_loss: 4.1718 - val_acc: 0.7375
Epoch 3/10
 - 21s - loss: 0.0030 - acc: 0.6737 - val_loss: 4.2377 - val_acc: 0.7250
Epoch 4/10
 - 22s - loss: 0.0026 - acc: 0.7365 - val_loss: 3.6695 - val_acc: 0.7375
Epoch 5/10
 - 22s - loss: 0.0020 - acc: 0.8018 - val_loss: 4.2323 - val_acc: 0.7375
Epoch 6/10
 - 21s - loss: 0.0015 - acc: 0.8679 - val_loss: 3.4565 - val_acc: 0.7750
Epoch 7/10
 - 21s - loss: 0.0010 - acc: 0.9095 - val_loss: 4.2464 - val_acc: 0.7250
Epoch 8/10
 - 21s - loss: 9.0159e-04 - acc: 0.9188 - val_loss: 3.9211 - val_acc: 0.7500
Epoch 9/10
 - 22s - loss: 6.2783e-04 - acc: 0.9531 - val_loss: 3.4251 - val_acc: 0.7875
Epoch 10/10
 - 22s - loss: 5.4477e-04 - acc: 0.9571 - val_loss: 3.4251 - val_acc: 0.7875
processing time: 215.00169444084167 seconds


Checking its score

In [19]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 3.42509536743
Test accuracy: 0.7875


Function to prepare data for prediction

In [20]:
words1 = []
def prepare_for_prediction(x):
    w1 = nltk.word_tokenize(x)
    # add to our words list
    words1.extend(w1)
    bag1 = []
    # list of tokenized words for the pattern
    pattern_words = words1
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag1.append(1) if w in pattern_words else bag1.append(0)
    bag1=np.asarray(bag1)
    bag2=[bag1]
    bag2=np.asarray(bag2)
    #print(len(bag1))
    return bag2

The example to predict

In [24]:
x=prepare_for_prediction("   1'' Drive Impact Sockets SUNEXpress Warranty Program Hammers & Needle Scalers 3/8'' Drive Impact Socket Sets Under Hoist Transmission Jacks Drive Tools 1/2'' Drive Impact Socket Sets 3/8'' Drive Impact Sockets #5 Spline Drive Impact Sockets Support 30080 - 3/8'' Drive 80 Tooth Low Profile Chrome Ratchet Distributor Sign In Wrench Sets Hex Driver Sets Lifting Equipment View Product Facebook Meet the Girls! New Pin Up Series 1-1/2'' Drive Impact Sockets Learn More About our Warranty Programs Picks, Hooks, Scrapers Contact Grinders Youtube Pry Bars Creepers Terms & Conditions Social Media go top Sanders & Polishers Under Hoist & Trailer Support Stands Magnetic Parts Trays Spray Guns RingoFire Digital 1/4'' Drive Impact Socket Sets 0 877.786.3939 Gear Store Chrome Socket Sets Pullers Sign Up for our Newsletter Shop Equipment Extension Sets 4412 - 12 Ton Bottle Jack Instagram Privacy Policy Rebates Buy Now Individual Wrenches 40th Anniversary Bit & Socket Sets Truck Equipment 1.800.833.7869 Careers Jack Stands Impact Wrenches BTSPEAKER - Bluetooth Socket Speaker Riveters Adapters Nut Drivers Engine Stands Adapter Sets Site Map 3/4'' Drive Impact Sockets Reducers & Super Reducers Portable Hydraulic Power Kits Specialty Sockets Specialty Tools SUNEXpress Videos Bottle Jacks Warranties 3315SR - 3/8'' Drive 15 Piece Metric Low Profile Impact Socket Set Impact Socket Sets Pliers ATV & Motorcycle Lift Sockets Hand Tools 9735 - 16 Piece Quick Change Bit Set 1'' Drive Impact Socket Sets Browse Online Now Floor Model Transmission Jacks Extensions & Adapters 1/4'' Drive Impact Sockets Presses & Accessories Download our Catalog Individual Sockets Introducing the New Chrome Socket Sets Shop Seats Floor Service Jacks Tool Storage About Us Bench Tools & Drill Presses Saws & Cut Off Tools 8035XTLG - 3 Drawer Service Cart with Sliding Top - Lime Green Screwdrivers Chrome Hand Tools Twitter 3/4'' Drive Impact Socket Sets Air Tool Parts & Accessories Torque & Measuring Tools Truck Jacks & Portable Air Lift Jack Ratchets 8057CAMO - Premium Full Drawer Service Cart- Camo Hand Tool Sets Web Outlets Registration Drills 1/2'' Drive Impact Sockets The Right Socket for Every Job Career Opportunities Air Tools Catalogs Engine Crane & Support Tools 1889 - 1/4'', 3/8'', and 1/2'' Drive 89 Piece SAE and Metric Master Chrome Socket Set Warranty Programs Service 877 786 3939 Toll Free 1 800 833 7869 Buy Now Distributor Sign In Warranties SUNEXpress Videos Contact Sockets Chrome Socket Sets Impact Socket Sets 1 4'' Drive Impact Socket Sets 3 8'' Drive Impact Socket Sets 1 2'' Drive Impact Socket Sets 3 4'' Drive Impact Socket Sets 1'' Drive Impact Socket Sets Extensions Adapters Adapter Sets Extension Sets Reducers Super Reducers Hex Driver Sets Specialty Sockets Individual Sockets 1 4'' Drive Impact Sockets 3 8'' Drive Impact Sockets 1 2'' Drive Impact Sockets 3 4'' Drive Impact Sockets 1'' Drive Impact Sockets 1 1 2'' Drive Impact Sockets 5 Spline Drive Impact Sockets Hand Tools Wrench Sets Hand Tool Sets Bit Socket Sets Pliers Pry Bars Screwdrivers Picks Hooks Scrapers Nut Drivers Chrome Hand Tools Drive Tools Torque Measuring Tools Individual Wrenches Air Tools Drills Grinders Hammers Needle Scalers Impact Wrenches Ratchets Riveters Sanders Polishers Saws Cut Off Tools Spray Guns Specialty Tools Air Tool Parts Accessories Lifting Equipment Adapters ATV Motorcycle Lift Bottle Jacks Engine Crane Support Tools Engine Stands Floor Model Transmission Jacks Floor Service Jacks Jack Stands Portable Hydraulic Power Kits Truck Equipment Truck Jacks Portable Air Lift Jack Under Hoist Trailer Support Stands Under Hoist Transmission Jacks Shop Equipment Bench Tools Drill Presses Creepers Presses Accessories Magnetic Parts Trays Pullers Shop Seats Tool Storage Support 40th Anniversary Catalogs Warranty Programs SUNEXpress Warranty Program Rebates Career Opportunities Menu Sockets Chrome Socket Sets Impact Socket Sets 1 4'' Drive Impact Socket Sets 3 8'' Drive Impact Socket Sets 1 2'' Drive Impact Socket Sets 3 4'' Drive Impact Socket Sets 1'' Drive Impact Socket Sets Extensions Adapters Adapter Sets Extension Sets Reducers Super Reducers Hex Driver Sets Specialty Sockets Individual Sockets 1 4'' Drive Impact Sockets 3 8'' Drive Impact Sockets 1 2'' Drive Impact Sockets 3 4'' Drive Impact Sockets 1'' Drive Impact Sockets 1 1 2'' Drive Impact Sockets 5 Spline Drive Impact Sockets Hand Tools Wrench Sets Hand Tool Sets Bit Socket Sets Pliers Pry Bars Screwdrivers Picks Hooks Scrapers Nut Drivers Chrome Hand Tools Drive Tools Torque Measuring Tools Individual Wrenches Air Tools Drills Grinders Hammers Needle Scalers Impact Wrenches Ratchets Riveters Sanders Polishers Saws Cut Off Tools Spray Guns Specialty Tools Air Tool Parts Accessories Lifting Equipment Adapters ATV Motorcycle Lift Bottle Jacks Engine Crane Support Tools Engine Stands Floor Model Transmission Jacks Floor Service Jacks Jack Stands Portable Hydraulic Power Kits Truck Equipment Truck Jacks Portable Air Lift Jack Under Hoist Trailer Support Stands Under Hoist Transmission Jacks Shop Equipment Bench Tools Drill Presses Creepers Presses Accessories Magnetic Parts Trays Pullers Shop Seats Tool Storage Support 40th Anniversary Catalogs Warranty Programs SUNEXpress Warranty Program Rebates Career Opportunities Search for Warranty Programs Home Support Warranty Programs Important Note We recommend that you purchase Sunex (r) Tools product from an Authorized Distributor to ensure warranty support and service Sunex(r) Tools assumes no service liability for product purchased from an Unauthorized Distributor To register your Sunex (r) product click here To find Authorized Distributors please refer to our Distributor Locator For additional help with parts and service please visit Service SunexTools com Sunex (r) Tools Inc Warrants to its customers that the company ' s Sunex (r) Tools branded products are free from defects in workmanship and materials Sunex (r) Tools Inc will repair or replace its Sunex (r) Tools branded products which fail to give satisfactory service due to defect in workmanship or materials based upon the Terms and Conditions of the Warranty plans attributed to that specific product This Warranty does not cover damage to equipment or tools arising from alteration abuse misuse damage and does not cover any repairs or replacement made by anyone other than Sunex (r) Tools or its Authorized Service Centers Return equipment or tools to Sunex (r) Tools or an Authorized Sunex(r) Warranty Center SHIPPING PREPAID Be certain to include your name address and contact information proof of purchase date and a description of the suspected defect The foregoing obligation is Sunex (r) Tools ' sole liability under this or any implied warranty and under no circumstances shall Sunex (r) Tools be liable for any incidental or consequential damages Click Here for SUNEXpress Warranty Warranty Policy 2016 Impact Sockets Sunex (r) Impact Sockets are warranted against defects in materials and workmanship for the life of the product This includes driveline sockets and super reducers This warranty does not apply to items that have been misused abused altered in any way or worn out from use Some Adapters 1805 2301 3305 Reducers 4301 5303 5524 6332 and Extra Thin Wall Impact Sockets 2624 2617 2626 2619 2628 2621 2632 2622 2634 2623 27 6 2708 2710 2712 are sold with out of box warranty Super Reducers however are covered under the Lifetime Warranty for impact sockets Hand Tools Sunex (r) Hand Tools are warranted against defects in materials and workmanship for the life of the product This includes wrenches screwdrivers pliers bit and socket sets pry bars pullers and drive tools Sunex Tools (r) sockets are clearly marked with the material from which they are manufactured Sockets intended for use with hand operated tools are made from Chrome Vanadium Alloy Steel These sockets also under lifetime warranty but are not designed for use with impact tools of any kind Doing so can cause serious injury and or void the warranty on these sockets Sunex Tools (r) Torque Wrenches carry a 90 Day Warranty During this warranty period Sunex Tools (r) will repair or replace at our option any part or unit which proves to be defective in material or workmanship Bit sets including but not limited to 9726 9732 9729 9734 and 9735 carry a limited lifetime warranty which covers any defects in workmanship or materials for the life of the product This warranty does not cover failures due directly or indirectly to misuse abuse negligence accidents normal wear and tear improper repair alterations or lack of maintenance Lifting Equipment Sunex (r) Lifting Equipment carry a One Year Warranty During this warranty period Sunex Tools (r) will replace any unit which proves to be defective in material or workmanship All floor jacks 3 5 ton in capacity and under will be replaced under the SUNEXpress(tm) Warranty HD Air Tools Sunex (r) HD Air Tools carry a Two Year Warranty During this warranty period Sunex Tools (r) will repair or replace at our option any part or unit which proves to be defective in material or workmanship Air Tools Sunex (r) Aluminum Air Tools carry a One Year Warranty During this warranty period Sunex Tools (r) will replace any unit which proves to be defective in material or workmanship through the SUNEXpress(tm) Warranty Spray Guns Sunex (r) Spray Guns and Spray Gun Accessories carry a 90 Day Warranty During this warranty period Sunex Tools (r) will repair or replace at our option any part or unit which proves to be defective in material or workmanship HD Shop Presses Sunex (r) HD Shop Presses are Made in USA and carry a Two Year Warranty During this warranty period Sunex Tools (r) will repair or replace at our option any part or unit which proves to be defective in material or workmanship Shop Equipment Sunex (r) Bench Grinders and Drill Presses One Year Warranty During this warranty period Sunex Tools (r) will repair or replace at our option any part or unit which proves to be defective in material or workmanship Carts Creepers Carts Creepers carry a 1 Year Warranty During this warranty period Sunex Tools (r) will repair or replace at our option any part or unit which proves to be defective in material or workmanship In addition during this warranty period we will replace any wear and tear part through the SUNEXpress(tm) Warranty This includes gas shocks drawer slides handles and casters View Our Full Range of Products Browse Online Now 0 go top FEATURED PRODUCTS Get the Latest Updates from Sunex Tools SIGN UP FOR OUR NEWSLETTER Click Here Quick Links Gear Store About Us Warranties Careers Contact Web Outlets Registration Privacy Policy Terms Conditions Site Map Social Media Facebook Youtube Instagram Twitter Service 877 786 3939 Toll Free 1 800 833 7869 Mailing address PO Box 1233 Greenville SC 29608 (c) 2018 Sunex Tools Powered by RingoFire Digital Sign up to receive the latest info on new Sunex products special offers and more Name First Last Email Occupation occupation Auto Mechanic Diesel Mechanic Equipment Mechanic Contractor MRO Facility Maintenance Enthusiast DIY Other Opt In I want emails from Sunex CAPTCHA X CLOSE POPUP Sunex products are trusted for their design engineering we will continue to equip our customers with high quality tools at the best value Shop today")
model.predict(x)

array([[ 1.,  0.,  0.]], dtype=float32)